In [3]:
import pandas as pd
from pathlib import Path
from itertools import product
from tqdm import tqdm

In [4]:
data = pd.read_csv('../data/authentication/unbalanced/RecodGait_v2/standartized_unbalanced.csv')
data['user'] = data['user'].apply(lambda x: int(x))
data['session'] = data['session'].apply(lambda x: int(x))
data

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,accel-z-57,accel-z-58,accel-z-59,user,level_0,timestamp diff,accel-start-time,index,session,window
0,-0.380865,-0.490814,-0.138980,-0.540292,-1.029622,0.151683,-0.271362,0.093922,0.233053,0.885288,...,-4.849618,-1.033230,-0.527983,1,0.0,0.025,0.025,1.0,1,0.0
1,0.759736,6.453595,3.141979,-1.764701,0.312573,-1.741856,-0.636970,-0.619488,-0.731601,-0.936111,...,0.786765,2.035368,2.023028,1,60.0,0.025,1.525,1.0,1,1.0
2,0.090258,0.748776,0.118458,0.980351,0.895728,1.314015,3.890168,-0.596918,-1.771354,0.153991,...,0.217113,-0.324870,0.715738,1,120.0,0.025,3.025,1.0,1,2.0
3,1.180711,0.123304,-0.242732,-2.090366,-3.026408,-2.704246,-1.275486,-1.903105,-0.559540,1.222120,...,1.849099,1.107216,-2.741191,1,180.0,0.025,4.525,1.0,1,3.0
4,1.504804,-3.261596,-1.013031,-0.832169,6.022226,1.731795,-2.193926,0.393643,2.146832,-0.179159,...,2.350917,1.821752,0.778877,1,240.0,0.025,6.025,1.0,1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27725,4.407922,2.885707,-1.464764,-3.105162,0.814450,-0.823310,-0.953835,-0.914520,-1.152465,-1.445332,...,-0.089251,-0.728952,-3.189619,144,3351292.0,0.025,141.025,280.0,3,94.0
27726,-2.558267,-1.422138,0.176106,0.849109,-0.541233,-0.348314,-2.053597,-1.504994,2.248131,-0.465023,...,7.003553,2.891149,-2.005648,144,3351352.0,0.025,142.525,280.0,3,95.0
27727,-1.092603,-4.081088,0.190277,-0.199893,-2.203307,-1.567213,-0.137114,0.511935,0.349388,-2.119933,...,-1.870522,0.583068,-2.984589,144,3351412.0,0.025,144.025,280.0,3,96.0
27728,-1.597260,0.267185,0.074118,-0.914575,-0.358683,0.152376,-0.300467,4.109457,3.517375,-4.086155,...,-8.364679,-0.525942,0.958939,144,3351472.0,0.025,145.525,280.0,3,97.0


In [41]:
import numpy as np
import os
from settings import *
from itertools import product
def resultant_vector(train_set, test_set):
    features_train = np.sqrt(np.sum(train_set**2, axis=2))
    features_test = np.sqrt(np.sum(test_set**2, axis=2))
    return features_train, features_test

def generate_mframes_recodGait(path, subset, magnitude = False, users_valid=None):
	# return set with size (n_samples, frame_size, n_accelerations)
	print("-- LOADING DATA from "+path+subset)
	files = [path+subset+'/'+f for f in os.listdir(path+subset) if f[0] != '.' and not f.endswith('vector.txt')]
	print(files)

	users = [int(f.split('/')[-1].split('___')[0].split('u')[-1]) for f in files]
	# files = [f'u{u}___s{s}.csv' for u, s in product(users, range(1,6))]
	users_nsession = {u: len(np.where(users==u)[0]) for u in np.unique(users)}
	print(frame_size, n_accelerations)
	train_set = np.zeros((0, frame_size, n_accelerations))
	test_set = np.empty((0, frame_size, n_accelerations))
	train_meta = np.empty((0, dim_meta), dtype=int)
	test_meta = np.empty((0, dim_meta), dtype=int)
	frame_id = 0
	for idx, f in enumerate(files):
		f_meta = f.split('___')
		user = int(f_meta[0].split('/')[-1])
		if(not users_valid or (user in users_valid)):
			session = idx+1
			data = np.loadtxt(f, delimiter=',')
			data = data[500:-500]
			frames = [data[i:i+frame_size, :] for i in range(0,data.shape[0]-frame_size,step)]
			frames = np.dstack(frames)
			frames = np.rollaxis(frames,-1)
			user_id = np.full((frames.shape[0]), user)
			session_id = np.full((frames.shape[0]), session)
			meta = np.empty((frames.shape[0], dim_meta), dtype=int)
			meta[:, label_user] = user_id
			meta[:, label_session] = session_id
			meta[:, label_frame] = np.arange(frame_id, frame_id+frames.shape[0])
			frame_id += frames.shape[0]
			if((users_nsession[user] > 3 and int(f_meta[1][-1]) > 3) or (users_nsession[user] <= 3 and int(f_meta[1][-1]) == users_nsession[user])):
				print(test_set)
				test_set = np.vstack((test_set, frames))
				test_meta = np.vstack((test_meta, meta))
			else:
				print(train_set, frames.shape)
				train_set = np.vstack((train_set, frames))
				train_meta = np.vstack((train_meta, meta))			
	if(magnitude):
		m_train, m_test = resultant_vector(train_set, test_set)
		train_set = np.dstack((train_set, m_train))
		test_set = np.dstack((test_set, m_test))
	return train_set, train_meta, test_set, test_meta

In [44]:
data = np.loadtxt('../data/original/RecodGait_v2/RecodGait v2/raw_data/01___session1____data_accelerometer.txt', delimiter=',')
print(data.shape)
data = data[500:-500]
print(data.shape)
frames = [data[i:i+frame_size, :] for i in range(0,data.shape[0]-frame_size,step)]

(12035, 4)
(11035, 4)


In [42]:
generate_mframes_recodGait('../data/original/RecodGait_v2/RecodGait v2/', 'raw_data')

-- LOADING DATA from ../data/original/RecodGait_v2/RecodGait v2/raw_data
['../data/original/RecodGait_v2/RecodGait v2/raw_data/12___session1____data_accelerometer.txt', '../data/original/RecodGait_v2/RecodGait v2/raw_data/132___session1____data_accelerometer.txt', '../data/original/RecodGait_v2/RecodGait v2/raw_data/53___session1____data_accelerometer.txt', '../data/original/RecodGait_v2/RecodGait v2/raw_data/12___session3____data_accelerometer.txt', '../data/original/RecodGait_v2/RecodGait v2/raw_data/50___session1____data_accelerometer.txt', '../data/original/RecodGait_v2/RecodGait v2/raw_data/11___session9____data_accelerometer.txt', '../data/original/RecodGait_v2/RecodGait v2/raw_data/77___session3____data_accelerometer.txt', '../data/original/RecodGait_v2/RecodGait v2/raw_data/140___session1____data_accelerometer.txt', '../data/original/RecodGait_v2/RecodGait v2/raw_data/03___session1____data_accelerometer.txt', '../data/original/RecodGait_v2/RecodGait v2/raw_data/66___session3___

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 2, the array at index 0 has size 3 and the array at index 1 has size 4

In [5]:
# Group by user and create a column for every unique sessions
data_sessions = data.groupby(['user', 'session']).size().unstack(fill_value=0)
data_sessions['S-count'] = data_sessions.apply(lambda row: 5 - row.value_counts().get(0, 0), axis=1)
train_data_sessions = data_sessions[data_sessions['S-count'].isin([3,5])]
val_data_sessions = data_sessions[data_sessions['S-count'] == 4]
test_data_sessions = data_sessions[data_sessions['S-count'] == 2]

In [ ]:
train_user_ids = train_data_sessions.index
val_user_ids = val_data_sessions.index
test_user_ids = test_data_sessions.index

(Index([ 2,  5,  7, 11, 14, 17, 21, 25, 27, 29, 30, 34, 37, 39, 40, 42, 43, 46,
        52, 53, 54, 58, 77],
       dtype='int64', name='user'),
 Index([1, 3, 8, 9, 13, 22, 28, 32, 36, 50], dtype='int64', name='user'),
 Index([  4,  10,  12,  15,  16,  18,  19,  20,  23,  24,  26,  31,  33,  35,
         38,  41,  44,  47,  49,  51,  55,  56,  57,  59,  60,  61,  62,  64,
         65,  66,  67,  68,  70,  72,  73,  75,  76,  80,  89,  90,  99, 100,
        101, 103, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 125, 126, 127, 128, 129, 130, 131,
        132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 143, 144],
       dtype='int64', name='user'))

In [ ]:
train_data_sessions

session,1,3,5,7,9,S-count
user,,,,,,
4,99,99,0,0,0,2
10,99,99,0,0,0,2
12,99,99,0,0,0,2
15,99,99,0,0,0,2
16,99,99,0,0,0,2
...,...,...,...,...,...,...
139,99,99,0,0,0,2
140,99,99,0,0,0,2
141,99,100,0,0,0,2


In [ ]:
val_data_sessions

AssertionError: 

In [ ]:
test_data_sessions

In [ ]:
def post_process_recodgait_dataset(input_path: Path, output_path: Path):
    # Read the input file
    data = pd.read_csv(input_path)
    # Preprocess the data
    data['user'] = data['user'].apply(lambda x: int(x))
    data['session'] = data['session'].apply(lambda x: int(x))
    # Group by user and create a column for every unique sessions
    data_sessions = data.groupby(['user', 'session']).size().unstack(fill_value=0)
    data_sessions['S-count'] = data_sessions.apply(lambda row: 5 - row.value_counts().get(0, 0), axis=1)
    # Separate the data in train, validation and test
    train_user_ids = data_sessions[data_sessions['S-count'].isin([3,5])].index
    val_user_ids = data_sessions[data_sessions['S-count'] == 4].index
    test_user_ids = data_sessions[data_sessions['S-count'] == 2].index
    train_df = data[data['user'].isin(train_user_ids)]
    val_df = data[data['user'].isin(val_user_ids)]
    test_df = data[data['user'].isin(test_user_ids)]
    return train_df

In [ ]:
def generate_df_from_pairs(
        pairs,
        data: pd.DataFrame,
        label: int,
        preffixes = ['accel-x', 'accel-y', 'accel-z'],
        additional_columns = ['user', 'session'],
        output_path: Path = None
    ):
    columns = [val for val in data.columns for preffix in preffixes if val.startswith(preffix)] + additional_columns
    # Extract the ids  
    pair_elem_1 = [val[0] for val in pairs]
    pair_elem_2 = [val[1] for val in pairs]
    # Extract the data from the pairs
    s1 = data.loc[pair_elem_1, columns].reset_index(drop=True)
    s2 = data.loc[pair_elem_2, columns].reset_index(drop=True)
    # Add a preffix to the columns
    s1 = s1.rename(lambda x: f'S1-{x}', axis=1)
    s2 = s2.rename(lambda x: f'S2-{x}', axis=1)
    # Concatenate the data and add the label
    result_df = pd.concat([s1, s2], axis=1)
    print(result_df.shape)
    result_df['label'] = label
    result_df.to_csv(output_path, index=False)
    # return result_df

In [ ]:
def create_positive_and_negative_pairs(data: pd.DataFrame, output_path: Path):
    # Identify the users in the dataset
    user_ids = data['user'].unique()
    # Generate the pairs
    pairs = []
    for user_id1, user_id2 in tqdm(product(user_ids, repeat=2), desc='User pairs', total=len(user_ids)**2):
        session_ids_user1 = data[data['user'] == user_id1]['session'].unique()
        session_ids_user2 = data[data['user'] == user_id2]['session'].unique()
        for session_user1, session_user2 in product(session_ids_user1, session_ids_user2):
            if session_user1 == session_user2 and user_id1 == user_id2:
                continue
            session_data_user1 = data[(data['user'] == user_id1) & (data['session'] == session_user1)]
            session_data_user2 = data[(data['user'] == user_id2) & (data['session'] == session_user2)]
            pair_type = '-'
            if user_id1 == user_id2:
                pair_type = '+'
            for idx_session_user_1, idx_session_user_2 in product(session_data_user1.index, session_data_user2.index):
                pairs.append(((idx_session_user_1, idx_session_user_2), pair_type))
    # Separate the pairs in positive and negative
    positive_pairs = [pair[0] for pair in pairs if pair[1] == '+']
    negative_pairs = [pair[0] for pair in pairs if pair[1] == '-']
    # Generate the dataframes
    generate_df_from_pairs(positive_pairs, data, 1, output_path=output_path/'positive.csv')
    generate_df_from_pairs(negative_pairs, data, 0, output_path=output_path/'negative.csv')

In [ ]:
train_df = post_process_recodgait_dataset(Path('../data/authentication/unbalanced/RecodGait_v2/standartized_unbalanced.csv'), None)

In [ ]:
create_positive_and_negative_pairs(train_df, output_path=Path('.'))

User pairs: 100%|██████████| 529/529 [00:48<00:00, 10.91it/s]


(1843382, 364)


In [ ]:
train_df_posneg.shape

In [ ]:
data_sessions = data.groupby(['user', 'session']).size().unstack(fill_value=0)
data_sessions.iloc[0].value_counts()

1
99    4
0     1
Name: count, dtype: int64

In [ ]:
train_data_sessions.shape, val_data_sessions.shape, test_data_sessions.shape

((23, 6), (10, 6), (82, 6))

In [ ]:
train_data_sessions.index

Index([ 2,  5,  7, 11, 14, 17, 21, 25, 27, 29, 30, 34, 37, 39, 40, 42, 43, 46,
       52, 53, 54, 58, 77],
      dtype='int64', name='user')

In [ ]:
train_user_ids = [int(i) for i in train_data_sessions.index]
test_user_ids = [int(i) for i in test_data_sessions.index]

In [ ]:
train_data_sessions.loc['MIN'] = train_data_sessions.max()
train_data_sessions

/tmp/ipykernel_472565/1474797423.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_sessions.loc['MIN'] = train_data_sessions.max()


session,1,3,5,7,9,TSessions,Samples,sessions_ids
user,,,,,,,,
1,99,99,99,99,0,4,396,"[1, 3, 5, 7]"
2,99,99,99,0,0,3,297,"[1, 3, 5]"
3,99,99,99,99,0,4,396,"[1, 3, 5, 7]"
5,99,99,99,99,99,5,495,"[1, 3, 5, 7, 9]"
7,99,99,99,0,0,3,297,"[1, 3, 5]"
8,99,99,99,99,0,4,396,"[1, 3, 5, 7]"
9,99,99,99,99,0,4,396,"[1, 3, 5, 7]"
11,99,99,99,99,99,5,495,"[1, 3, 5, 7, 9]"
13,99,99,99,99,0,4,396,"[1, 3, 5, 7]"


In [ ]:
train_data_sessions.loc['TOTAL'] = train_data_sessions.sum()
train_data_sessions

/tmp/ipykernel_472565/1213087852.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_sessions.loc['TOTAL'] = train_data_sessions.sum()


session,1,3,5,7,9,TSessions,Samples,sessions_ids
user,,,,,,,,
1,99,99,99,99,0,4,396,"[1, 3, 5, 7]"
2,99,99,99,0,0,3,297,"[1, 3, 5]"
3,99,99,99,99,0,4,396,"[1, 3, 5, 7]"
5,99,99,99,99,99,5,495,"[1, 3, 5, 7, 9]"
7,99,99,99,0,0,3,297,"[1, 3, 5]"
8,99,99,99,99,0,4,396,"[1, 3, 5, 7]"
9,99,99,99,99,0,4,396,"[1, 3, 5, 7]"
11,99,99,99,99,99,5,495,"[1, 3, 5, 7, 9]"
13,99,99,99,99,0,4,396,"[1, 3, 5, 7]"


In [ ]:
test_data_sessions.loc['MAX'] = test_data_sessions.max()
test_data_sessions

/tmp/ipykernel_472565/3417347460.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_sessions.loc['MAX'] = test_data_sessions.max()


session,1,3,5,7,9,TSessions,Samples,sessions_ids
user,,,,,,,,
4,99,99,0,0,0,2,198,"[1, 3]"
10,99,99,0,0,0,2,198,"[1, 3]"
12,99,99,0,0,0,2,198,"[1, 3]"
15,99,99,0,0,0,2,198,"[1, 3]"
16,99,99,0,0,0,2,198,"[1, 3]"
...,...,...,...,...,...,...,...,...
140,99,99,0,0,0,2,198,"[1, 3]"
141,99,100,0,0,0,2,199,"[1, 3]"
143,99,99,0,0,0,2,198,"[1, 3]"


In [ ]:
[i for i in test_data_sessions.columns if test_data_sessions.loc[4][i] != 0 ]

[1, 3, 'TSessions', 'Samples', 'sessions_ids']

In [ ]:
# Get the session ids for each user
test_data_sessions['sessions_ids'] = test_data_sessions.apply(lambda row: [i for i in test_data_sessions.columns if row[i] != 0 ], axis=1)
test_data_sessions

/tmp/ipykernel_472565/2837402989.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_sessions['sessions_ids'] = test_data_sessions.apply(lambda row: [i for i in test_data_sessions.columns if row[i] != 0 ], axis=1)


session,1,3,5,7,9,TSessions,Samples,sessions_ids
user,,,,,,,,
4,99,99,0,0,0,2,198,"[1, 3, TSessions, Samples, sessions_ids]"
10,99,99,0,0,0,2,198,"[1, 3, TSessions, Samples, sessions_ids]"
12,99,99,0,0,0,2,198,"[1, 3, TSessions, Samples, sessions_ids]"
15,99,99,0,0,0,2,198,"[1, 3, TSessions, Samples, sessions_ids]"
16,99,99,0,0,0,2,198,"[1, 3, TSessions, Samples, sessions_ids]"
...,...,...,...,...,...,...,...,...
140,99,99,0,0,0,2,198,"[1, 3, TSessions, Samples, sessions_ids]"
141,99,100,0,0,0,2,199,"[1, 3, TSessions, Samples, sessions_ids]"
143,99,99,0,0,0,2,198,"[1, 3, TSessions, Samples, sessions_ids]"


## TEST

In [ ]:
import random

In [ ]:
data_for_test = {
    user_id: {
        'S1': [int(val) for val in data[(data['user'] == user_id) & (data['session'] == 1)].index],
        'S3': [int(val) for val in data[(data['user'] == user_id) & (data['session'] == 3)].index],
    }
    for user_id in test_user_ids
}

In [ ]:
positive_pairs = []

pick_from_session = 99

for user_id in test_user_ids:
    s1_ids = data_for_test[user_id]['S1']
    s1_subset = random.sample(s1_ids, pick_from_session if pick_from_session > 1 else int(len(s1_ids)*pick_from_session))
    s3_ids = data_for_test[user_id]['S3']
    s3_subset = random.sample(s3_ids, pick_from_session if pick_from_session > 1 else int(len(s3_ids)*pick_from_session))
    for s1 in s1_subset:
        for s3 in s3_subset:
            positive_pairs.append((s1, s3))
    

In [ ]:
len(positive_pairs)

803682

In [ ]:
total_users = 82
samples_per_user = 99
total_users*samples_per_user*samples_per_user

803682

In [ ]:
negative_pairs = []

pick_from_session = 99
negative_per_positive = 5

for user_id in test_user_ids:
    impostor_ids = [val for val in test_user_ids if val != user_id]
    impostor_ids = random.sample(impostor_ids, negative_per_positive)
    s1_ids = data_for_test[user_id]['S1']
    s1_subset = random.sample(s1_ids, pick_from_session if pick_from_session > 1 else int(len(s1_ids)*pick_from_session))
    for impostor_id in impostor_ids:
        s3_ids = data_for_test[impostor_id]['S3']
        s3_subset = random.sample(s3_ids, pick_from_session if pick_from_session > 1 else int(len(s3_ids)*pick_from_session))
        for s1 in s1_subset:
            for s3 in s3_subset:
                negative_pairs.append((s1, s3))

In [ ]:
len(negative_pairs)

4018410

In [ ]:
len(negative_pairs)+len(positive_pairs)

4822092

In [ ]:
preffixes = ['accel-x', 'accel-y', 'accel-z']
columns = [f'{preffix}-{i}' for preffix in preffixes for i in range(0, 59)]
columns

['accel-x-0',
 'accel-x-1',
 'accel-x-2',
 'accel-x-3',
 'accel-x-4',
 'accel-x-5',
 'accel-x-6',
 'accel-x-7',
 'accel-x-8',
 'accel-x-9',
 'accel-x-10',
 'accel-x-11',
 'accel-x-12',
 'accel-x-13',
 'accel-x-14',
 'accel-x-15',
 'accel-x-16',
 'accel-x-17',
 'accel-x-18',
 'accel-x-19',
 'accel-x-20',
 'accel-x-21',
 'accel-x-22',
 'accel-x-23',
 'accel-x-24',
 'accel-x-25',
 'accel-x-26',
 'accel-x-27',
 'accel-x-28',
 'accel-x-29',
 'accel-x-30',
 'accel-x-31',
 'accel-x-32',
 'accel-x-33',
 'accel-x-34',
 'accel-x-35',
 'accel-x-36',
 'accel-x-37',
 'accel-x-38',
 'accel-x-39',
 'accel-x-40',
 'accel-x-41',
 'accel-x-42',
 'accel-x-43',
 'accel-x-44',
 'accel-x-45',
 'accel-x-46',
 'accel-x-47',
 'accel-x-48',
 'accel-x-49',
 'accel-x-50',
 'accel-x-51',
 'accel-x-52',
 'accel-x-53',
 'accel-x-54',
 'accel-x-55',
 'accel-x-56',
 'accel-x-57',
 'accel-x-58',
 'accel-y-0',
 'accel-y-1',
 'accel-y-2',
 'accel-y-3',
 'accel-y-4',
 'accel-y-5',
 'accel-y-6',
 'accel-y-7',
 'accel-y-8',

In [ ]:
columns = [val for val in data.columns for preffix in preffixes if val.startswith(preffix)]
data.loc[0, columns]

accel-x-0    -0.380865
accel-x-1    -0.490814
accel-x-2    -0.138980
accel-x-3    -0.540292
accel-x-4    -1.029622
                ...   
accel-z-55   -2.809098
accel-z-56   -5.147924
accel-z-57   -4.849618
accel-z-58   -1.033230
accel-z-59   -0.527983
Name: 0, Length: 180, dtype: float64

In [ ]:
def generate_csv_from_pairs(pairs, data: pd.DataFrame, label: int):
    pairs_data = []
    preffixes = ['accel-x', 'accel-y', 'accel-z']
    # columns = [f'{preffix}-{i}' for preffix in preffixes for i in range(0, 59)]
    columns = [val for val in data.columns if val.startswith(preffix) for preffix in preffixes]
    pair_elem_1 = [val[0] for val in pairs]
    pair_elem_2 = [val[1] for val in pairs]
    s1 = data.loc[pair_elem_1, columns]
    s2 = data.loc[pair_elem_2, columns]
    # Add a preffix to the columns
    s1 = s1.rename(lambda x: f'S1-{x}')
    s2 = s2.rename(lambda x: f'S2-{x}')
    pairs_data.append(pd.concat([s1, s2], axis=1))
    pairs_data['label'] = label
    return pd.DataFrame(pairs_data)

In [ ]:
from dummy_rg import create_dummy_rg_df
create_dummy_rg_df()

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-y-0,accel-y-1,accel-y-2,accel-y-3,accel-y-4,accel-z-0,accel-z-1,accel-z-2,accel-z-3,accel-z-4,user,session
0,1-1-1,1-1-1,1-1-1,1-1-1,1-1-1,1-1-1,1-1-1,1-1-1,1-1-1,1-1-1,1-1-1,1-1-1,1-1-1,1-1-1,1-1-1,1,1
1,1-1-2,1-1-2,1-1-2,1-1-2,1-1-2,1-1-2,1-1-2,1-1-2,1-1-2,1-1-2,1-1-2,1-1-2,1-1-2,1-1-2,1-1-2,1,1
2,1-2-1,1-2-1,1-2-1,1-2-1,1-2-1,1-2-1,1-2-1,1-2-1,1-2-1,1-2-1,1-2-1,1-2-1,1-2-1,1-2-1,1-2-1,1,2
3,1-2-2,1-2-2,1-2-2,1-2-2,1-2-2,1-2-2,1-2-2,1-2-2,1-2-2,1-2-2,1-2-2,1-2-2,1-2-2,1-2-2,1-2-2,1,2
4,1-3-1,1-3-1,1-3-1,1-3-1,1-3-1,1-3-1,1-3-1,1-3-1,1-3-1,1-3-1,1-3-1,1-3-1,1-3-1,1-3-1,1-3-1,1,3
5,1-3-2,1-3-2,1-3-2,1-3-2,1-3-2,1-3-2,1-3-2,1-3-2,1-3-2,1-3-2,1-3-2,1-3-2,1-3-2,1-3-2,1-3-2,1,3
6,1-4-1,1-4-1,1-4-1,1-4-1,1-4-1,1-4-1,1-4-1,1-4-1,1-4-1,1-4-1,1-4-1,1-4-1,1-4-1,1-4-1,1-4-1,1,4
7,1-4-2,1-4-2,1-4-2,1-4-2,1-4-2,1-4-2,1-4-2,1-4-2,1-4-2,1-4-2,1-4-2,1-4-2,1-4-2,1-4-2,1-4-2,1,4
8,2-1-1,2-1-1,2-1-1,2-1-1,2-1-1,2-1-1,2-1-1,2-1-1,2-1-1,2-1-1,2-1-1,2-1-1,2-1-1,2-1-1,2-1-1,2,1
9,2-1-2,2-1-2,2-1-2,2-1-2,2-1-2,2-1-2,2-1-2,2-1-2,2-1-2,2-1-2,2-1-2,2-1-2,2-1-2,2-1-2,2-1-2,2,1


## TRAIN

In [ ]:
from itertools import combinations

In [ ]:
data_for_train = {
    user_id: {
        f'S{session}' : [int(val) for val in data[(data['user'] == user_id) & (data['session'] == session)].index]
        for session in train_data_sessions.loc[user_id]['sessions_ids']
    }
    for user_id in train_user_ids
}

In [ ]:
train_user_ids

[2,
 3,
 5,
 7,
 8,
 9,
 11,
 13,
 14,
 17,
 21,
 22,
 25,
 27,
 28,
 29,
 30,
 32,
 34,
 36,
 37,
 39,
 40,
 42,
 43,
 46,
 50,
 52,
 53,
 54,
 58,
 77,
 1]

In [ ]:
positive_pairs = []

pick_from_session = 99

for user_id in train_user_ids:
    sessions_ids = train_data_sessions.loc[user_id,'sessions_ids']
    for first_session_id, second_session_id in combinations(sessions_ids, 2):
        first_ids = data_for_train[user_id][f'S{first_session_id}']
        first_subset = random.sample(first_ids, pick_from_session if pick_from_session > 1 else int(len(first_ids)*pick_from_session))
        second_ids = data_for_train[user_id][f'S{second_session_id}']
        second_subset = random.sample(second_ids, pick_from_session if pick_from_session > 1 else int(len(second_ids)*pick_from_session))
        for first in first_subset:
            for second in second_subset:
                positive_pairs.append((first, second))
    

In [ ]:
len(positive_pairs)

1470150

In [ ]:
1470150*5

7350750

In [ ]:
data.loc[data_for_test[4]['S3']]

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,accel-z-57,accel-z-58,accel-z-59,timestamp diff,user,index,session,level_0,accel-start-time,window
1188,-1.653561,-2.519575,-1.506114,-2.099624,-2.136091,-1.479297,-1.038491,-1.129791,-0.215552,0.512291,...,2.312264,2.792678,2.056905,0.025,4,13.0,3,143904.0,0.025,0.0
1189,0.974226,0.235028,-0.745470,-0.552413,-0.665351,0.079677,0.879018,-2.329216,-1.653362,-3.341059,...,2.685338,-1.122210,0.412699,0.025,4,13.0,3,143964.0,1.525,1.0
1190,-3.659260,-1.051734,2.869330,3.409195,3.252914,3.361076,3.208949,0.638311,4.774703,1.430671,...,5.256629,0.988531,5.836710,0.025,4,13.0,3,144024.0,3.025,2.0
1191,-4.089611,3.299361,-1.000038,0.121797,-3.807741,-4.010466,-3.623865,-3.358871,-2.184408,2.739828,...,4.387108,2.128408,-0.786076,0.025,4,13.0,3,144084.0,4.525,3.0
1192,3.456360,2.694654,2.717168,2.270423,2.410555,4.319924,2.132691,5.935977,-4.455462,-3.374960,...,-3.644356,0.148269,0.847316,0.025,4,13.0,3,144144.0,6.025,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1282,-2.363038,-1.190185,0.564062,7.085169,1.371091,-6.152905,-0.995118,0.190143,1.152311,0.835250,...,0.619966,0.910285,-1.834514,0.025,4,13.0,3,149544.0,141.025,94.0
1283,4.215384,2.445118,1.636924,-0.075621,1.834894,2.712164,-7.186368,2.251249,-0.794391,-0.528737,...,1.593775,-1.925333,3.814876,0.025,4,13.0,3,149604.0,142.525,95.0
1284,-2.765242,-3.438244,-2.891765,1.743369,-1.197367,4.103485,5.309978,-5.565059,-3.519667,1.480737,...,4.762945,1.835405,1.872553,0.025,4,13.0,3,149664.0,144.025,96.0
1285,0.492617,5.517211,5.127559,-0.478673,3.345488,2.096338,0.272276,-5.066971,1.277972,2.296954,...,-1.451410,3.264199,-2.817852,0.025,4,13.0,3,149724.0,145.525,97.0


In [ ]:
data_for_test

{4: {'S1': [1089,
   1090,
   1091,
   1092,
   1093,
   1094,
   1095,
   1096,
   1097,
   1098,
   1099,
   1100,
   1101,
   1102,
   1103,
   1104,
   1105,
   1106,
   1107,
   1108,
   1109,
   1110,
   1111,
   1112,
   1113,
   1114,
   1115,
   1116,
   1117,
   1118,
   1119,
   1120,
   1121,
   1122,
   1123,
   1124,
   1125,
   1126,
   1127,
   1128,
   1129,
   1130,
   1131,
   1132,
   1133,
   1134,
   1135,
   1136,
   1137,
   1138,
   1139,
   1140,
   1141,
   1142,
   1143,
   1144,
   1145,
   1146,
   1147,
   1148,
   1149,
   1150,
   1151,
   1152,
   1153,
   1154,
   1155,
   1156,
   1157,
   1158,
   1159,
   1160,
   1161,
   1162,
   1163,
   1164,
   1165,
   1166,
   1167,
   1168,
   1169,
   1170,
   1171,
   1172,
   1173,
   1174,
   1175,
   1176,
   1177,
   1178,
   1179,
   1180,
   1181,
   1182,
   1183,
   1184,
   1185,
   1186,
   1187],
  'S3': [1188,
   1189,
   1190,
   1191,
   1192,
   1193,
   1194,
   1195,
   1196,
   1197,
   

In [ ]:
# Generate the test samples
for key, group in data[data['user'].isin(test_data_sessions.index)].groupby(['user', 'session']):
    print(key)

(4, 1)
(4, 3)
(10, 1)
(10, 3)
(12, 1)
(12, 3)
(15, 1)
(15, 3)
(16, 1)
(16, 3)
(18, 1)
(18, 3)
(19, 1)
(19, 3)
(20, 1)
(20, 3)
(23, 1)
(23, 3)
(24, 1)
(24, 3)
(26, 1)
(26, 3)
(31, 1)
(31, 3)
(33, 1)
(33, 3)
(35, 1)
(35, 3)
(38, 1)
(38, 3)
(41, 1)
(41, 3)
(44, 1)
(44, 3)
(47, 1)
(47, 3)
(49, 1)
(49, 3)
(51, 1)
(51, 3)
(55, 1)
(55, 3)
(56, 1)
(56, 3)
(57, 1)
(57, 3)
(59, 1)
(59, 3)
(60, 1)
(60, 3)
(61, 1)
(61, 3)
(62, 1)
(62, 3)
(64, 1)
(64, 3)
(65, 1)
(65, 3)
(66, 1)
(66, 3)
(67, 1)
(67, 3)
(68, 1)
(68, 3)
(70, 1)
(70, 3)
(72, 1)
(72, 3)
(73, 1)
(73, 3)
(75, 1)
(75, 3)
(76, 1)
(76, 3)
(80, 1)
(80, 3)
(89, 1)
(89, 3)
(90, 1)
(90, 3)
(99, 1)
(99, 3)
(100, 1)
(100, 3)
(101, 1)
(101, 3)
(103, 1)
(103, 3)
(104, 1)
(104, 3)
(106, 1)
(106, 3)
(107, 1)
(107, 3)
(108, 1)
(108, 3)
(109, 1)
(109, 3)
(110, 1)
(110, 3)
(111, 1)
(111, 3)
(112, 1)
(112, 3)
(113, 1)
(113, 3)
(114, 1)
(114, 3)
(115, 1)
(115, 3)
(116, 1)
(116, 3)
(117, 1)
(117, 3)
(118, 1)
(118, 3)
(119, 1)
(119, 3)
(120, 1)
(120, 3)
(121